<a href="https://colab.research.google.com/github/agustin-sarasua/workshop-categorization/blob/master/workshop_categorizacion_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop Categorizacion 
Items categorization with NLP, Keras, Tensorflow, Pandas and Numpy

In [0]:
import requests
import time
import json
from collections import Counter
import random
from IPython.display import Image
# https://github.com/sepandhaghighi/pycm
!pip install pycm==1.8

import matplotlib.pyplot as plt
plt.figure(figsize=(12,9))

import pandas as pd # data preprocessing
import numpy as np # linear algebra

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

# Gather the Data

Download the zipped dataset of items from Google Drive (using the shareable link) and uncompress it.

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


download_file_from_google_drive('1kSPUjkf7gkB77UVTNnIW_cmGIjsa7GSh', 'arq.png')                
download_file_from_google_drive('1lD7fMlzNhx0AaPkoksrqJ4HEiTL_u5KM', 'vec.png')

download_file_from_google_drive('1Ql0az0GAC1_yHpHsAFvspKt1HZGMZOOa', 'cellphones.zip')
!unzip 'cellphones.zip'

Here is the architecture of the Neural Network that we are going to train

In [0]:
display(Image('arq.png'))

## Load the dataset using pandas
The dataset is a csv tab separated

See: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.htm

In [0]:
# 1 LoC aprox.
def read_csv(path='cellphones.csv'):
  dataset = # Your code here
  return dataset

df = read_csv()

# Explore the Data

In [0]:
#df.describe()
#df.head()

Find how many classes we have in the dataset. 

This classes will be the output of the Neural Network

In [0]:
# 1 LoC aprox.
def get_num_classes(df):
  output_length = # Your code here
  return output_length

num_classes = get_num_classes(df)
print("Output Lenght:", num_classes)
print("# Training examples:", len(df))

### Visualize the # of items per class

Bar plot the number of items per class.
1 - before ploting sort de values by counts.

See: 
Pandas Groupby https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

Pandas Count https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.count.html

Pandas Sort Values http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values

Pandas plot bar https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.bar.html



In [0]:
# From 1 to 4 LoC aprox. 
def plot_items_per_class(df):
  # Your code here

plot_items_per_class(df)  

### Visualize the class distribution

Plot the histogram - A histogram is a representation of the distribution of data

See: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.hist.html

In [0]:
# From 1 to 4 LoC aprox. 
# Almost identical to previos solution
def plot_distribution(df):
  # Your code here

plot_distribution(df)  

# Prepare the data for training
Here is an image showing what we are to do with the data


In [0]:
display(Image('vec.png'))

## Tokenization and Vectorization

Tokenization: Divide the texts into words or smaller sub-texts (tokens). This determines the “vocabulary” of the dataset.

Vectorization: Define a good numerical measure to characterize these texts.

Steps:
1. Fit a Keras Tokenizer using the text corpus (all the titles in the dataset)
2. Build a dictionary of indexes -> tokens
3. Create a function that parse the text to a sequence of decimals representing each token (step 2 in the image presented)
4. Vectorize the text using different representations or techniques (tfidf, bow, binary, etc)
5. Vectorize the labels (outputs of the NN)
6. Build a dictionary of indexes -> labels


### Fit a Keras Tokenizer

**From** https://keras.io/preprocessing/text/:

This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...

In [0]:
# 2 LoC aprox.
def fit_keras_tokenizer(corpus, num_features=5000):
  tokenizer = # Your code here
  # Your code here (remember to call fit_on_texts)
  return tokenizer

In [0]:

tic = time.time()
num_features=5000

tokenizer = fit_keras_tokenizer(df['title'].values, num_features)

toc = time.time()    
print("Time to fit tokenizer: " + str(1000*(toc-tic)) + " ms")


### Build dictionary of index -> token

It shoud return a dictionary with the following format: 

{..., 1: "con", 2: "tablet", ...}

Note: use tokenizer.word_index

In [0]:
# 1 LoC aprox. Could be more
def build_idx_to_token(tokenizer):
  idx_to_tkn = # Your code here
  return idx_to_tkn

idx_to_token = build_idx_to_token(tokenizer)
print(idx_to_token)

### Parse titles to sequence of indexes
This function returns a sequence of indices representing the text

See https://keras.io/preprocessing/sequence/:

sequence.pad_sequences

use: tokenizer.texts_to_sequences(text)

In [0]:
'''
text: list of titles
max_seq_length: fixed length to the titles. 
  If the title has more than "max_seq_length" tokens => truncate it to max_seq_length
  If the title has less than "max_seq_length" tokens => complete at the begining with 0's 
'''
# 2 LoC aprox.
def parse_title_to_sequences(text, tokenizer, max_seq_length=12):
  # 1 - text to sequences
  text_sequences = # Your code here
  # 2 - Pad the sequences to a fix length, so every example has the same length
  text_sequences = # Your code here
  return text_sequences

Visualize what the vectorizer is actually doing

In [0]:
def print_vectorized_title(title, tokenizer, idx_to_token, num_features, mode='count'):
  print("------------------")
  print("Title:", title)
  print("------------------")
  print("Vectorizing title using %s mode" % mode)
  print("------------------")
  text_sequences = parse_title_to_sequences([title], tokenizer)

  vectorized_title = tokenizer.sequences_to_matrix(text_sequences.tolist(), mode=mode)
  assert vectorized_title.shape == (1, num_features)

  for tkn in set(text_sequences[0]):
    if tkn > 0:
      print(idx_to_token[tkn] ,vectorized_title[0][tkn])

title = "tablet samsung de tablet con para pantalla tablet"
print_vectorized_title(title, tokenizer, idx_to_token, num_features, 'binary')
print_vectorized_title(title, tokenizer, idx_to_token, num_features, 'count')
print_vectorized_title(title, tokenizer, idx_to_token, num_features, 'tfidf')


### Vectorize the text and labels

This function append a new column called "label" with an integer that uniquely identify that domain_id.
See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html


In [0]:
# 1 LoC aprox
def parse_labels_to_decimal(df):
  df['label'] = # Your code here
parse_labels_to_decimal(df)  

This builds the labels vocabulary. It shoud return a dictionary with the following format: 
{..., 1: "TABLETS", 2: "CELLPHONES", ...}

In [0]:
# 1 LoC aprox. Could be more
def build_idx_to_label(df):
  idx_to_label = # Your code here
  return idx_to_label

idx_to_label = build_idx_to_label(df)
# Check that everything is consistent
assert df[df['label']==3].reset_index().iloc[0]['domain_id'] == idx_to_label[3]
print(idx_to_label)

We will be using categorical_crossentropy loss function (softmax), for this reason, we need to use one_hot encoding for our labels.

The representation should look like: [0, 0, ..., 1, 0] sparse vector with a "1" in the position of the target label.
  
 See https://keras.io/utils/ (to_categorial function)

In [0]:
from tensorflow.python.keras.utils import to_categorical

# 1 LoC aprox
def vectorize_labels(data, num_labels):
  # Convert labels to categorical one-hot encoding
  one_hot_labels = # Your code here
  return one_hot_labels

labels = vectorize_labels(df['label'].values, len(set(df['label'].values)))
# Run some Checks 
assert df.iloc[10]['domain_id'] == idx_to_label[df.iloc[10]['label']]
assert labels.shape[1] == num_classes

## Split the dataset

We will use 80% for training and 20% for validation

See: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [0]:
from sklearn.model_selection import train_test_split

# 1 LoC aprox.
def split_and_shuffle_dataset(df, val_size=0.2):
  # Random shuffle the dataset
  df = df.sample(frac=1).reset_index(drop=True)
  df_train, df_val = # Your code here
  return df_train.reset_index(drop=True), df_val.reset_index(drop=True)

df_train, df_val = split_and_shuffle_dataset(df)
print(df_train.shape)
print(df_val.shape)

Vectorize the datasets 

In [0]:
def vectorize_the_datasets(df, df_train, df_val, tokenizer, mode='tfidf'):
  
  x_train_seq = parse_title_to_sequences(df_train['title'].values, tokenizer)
  x_train_vec = tokenizer.sequences_to_matrix(x_train_seq.tolist(), mode=mode)
  y_train_vec = vectorize_labels(df_train['label'].values, len(df.domain_id.unique()))

  x_val_seq = parse_title_to_sequences(df_val['title'].values, tokenizer)
  x_val_vec = tokenizer.sequences_to_matrix(x_val_seq.tolist(), mode=mode)
  y_val_vec = vectorize_labels(df_val['label'].values, len(df.domain_id.unique()))
  
  return x_train_vec, y_train_vec, x_val_vec, y_val_vec

x_train, y_train, x_val, y_val = vectorize_the_datasets(df, df_train, df_val, tokenizer, 'tfidf')  

# Run some checks
index = 45
assert y_train[index][df_train.iloc[index]['label']] == 1.
assert y_val[index][df_val.iloc[index]['label']] == 1.

# Build the model
We will first create a model using Keras Sequential API

Learn More: https://keras.io/getting-started/sequential-model-guide/

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam

def build_model(num_features, num_classes):
  model = Sequential([
      Dense(512, input_shape=(num_features,), activation='relu'),
      Dense(num_classes, activation='softmax'),
  ])

  # For a multi-class classification problem
  model.compile(optimizer=Adam(lr=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

model = build_model(x_train.shape[1], y_train.shape[1])

### Visualizing the Neural Network

In [0]:
!apt install graphviz;
!pip install pydot pydot-ng;

In [0]:
from keras.utils import plot_model

# Model summary
print(model.summary())

# Plot model graph
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
display(Image('model.png'))

# Train the model
It is time to train the model

See: https://keras.io/models/model/ (fit function)

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

def train_model(x_train, y_train, x_val, y_val):
  # Create callback for early stopping on validation loss. If the loss does
  # not decrease in two consecutive tries, stop training.
  callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

  # Train and validate model.
  history = model.fit(
      x_train,
      y_train,
      callbacks=callbacks,
      epochs=20,
      validation_data=(x_val, y_val),
      verbose=1,  # Logs once per epoch.
      batch_size=128)
  return history

history = train_model(x_train, y_train, x_val, y_val)

### Visualize the learning curves

In [0]:
import matplotlib.pyplot as plt

def visualize_training_history(history):
  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

visualize_training_history(history)  

## Evaluate the model

Write a function that takes any datasets (train or validation or any other), makes a prediction and return the decimal representation of the label

See: model.predict function https://keras.io/models/model/

See: np.argmax function https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html


In [0]:
# 2 LoC aprox.
def evaluate_model(x_data, model):
  y_pred = # Your code here
  y_pred = # Your code here
  return y_pred

y_pred = evaluate_model(x_val, model)
y_act = np.argmax(y_val, axis=1)

### Visualize the errors and metrics

In [0]:
# From https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
from sklearn.metrics import f1_score
from pycm import *

cm = ConfusionMatrix(actual_vector=y_act, predict_vector=y_pred)
cm.save_html("cm_html")
#cm.print_matrix()
#cm.print_normalized_matrix()
#cm.print_matrix(one_vs_all=True, class_name=2)

print("Global F1-Score:" , f1_score(y_act, y_pred, average='micro'))

In [0]:
def print_errors(df_val, y_act, y_pred, idx_to_label, class_label=None, pred_label=None):
  for idx, row in df_val.iterrows():
    if class_label is not None and pred_label is not None:
      if y_act[idx] != y_pred[idx] and row['label']==class_label and y_pred[idx] == pred_label:
        print(row['item_id'], row['title'], 'Truth:', row['domain_id'], 'Prediction:',idx_to_label[y_pred[idx]])
    else:
      if y_act[idx] != y_pred[idx]:
        print(row['item_id'], row['title'], 'Truth:', row['domain_id'], 'Prediction:',idx_to_label[y_pred[idx]])

print_errors(df_val, y_act, y_pred, idx_to_label, 17, 8)        

In [0]:
print("Y_act", y_act[:10])
print("Y_pred",y_pred[:10])
print("Title:", df_val.iloc[0]['title'])
assert df_val.iloc[1]['domain_id'] == idx_to_label[y_pred[1]]


That is all for now! Thank you very much and be happy!